In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

In [2]:
# tool create 

@tool
def multiply(a:int, b:int)->int:
    "Given two numbers a and b this returns their product"
    return a*b

In [3]:
print(multiply.invoke({'a': 3, 'b':5}))

15


In [4]:
llm = ChatOpenAI()

Tool binding

In [5]:
llm_with_tools = llm.bind_tools([multiply])

In [6]:
llm_with_tools.invoke('Hi how are you?')

AIMessage(content="Hello! I'm just a computer program, so I don't have feelings, but I'm here to help you. How can I assist you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 57, 'total_tokens': 89, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-ChaSYqIalq2AOmWXs9KNX712NPFmn', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--19c92b25-436c-4ec2-a018-6db152b5b576-0', usage_metadata={'input_tokens': 57, 'output_tokens': 32, 'total_tokens': 89, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [8]:
result = llm_with_tools.invoke('Can you multiply 3 with 10')

In [10]:
result.tool_calls[0]

{'name': 'multiply',
 'args': {'a': 3, 'b': 10},
 'id': 'call_82wW3ugypZOv8Us7J6asJXyd',
 'type': 'tool_call'}

Tool calling

In [11]:
multiply.invoke(result.tool_calls[0]['args'])

30

In [12]:
multiply.invoke(result.tool_calls[0])

ToolMessage(content='30', name='multiply', tool_call_id='call_82wW3ugypZOv8Us7J6asJXyd')

Better implementation

In [19]:
query = HumanMessage( 'Can you multiply 3 with 10')

In [28]:
messages = [query]

In [31]:
messages

[HumanMessage(content='Can you multiply 3 with 10', additional_kwargs={}, response_metadata={})]

In [32]:
result = llm_with_tools.invoke(messages)
messages.append(result)


In [33]:
messages

[HumanMessage(content='Can you multiply 3 with 10', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 60, 'total_tokens': 77, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Chb5y3hondwokRDYHlW4SKvchzScB', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--c94f5d86-f688-421e-8123-f15bcb25583d-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 10}, 'id': 'call_eg3zlbbPrLNSEZodvarWtn7X', 'type': 'tool_call'}], usage_metadata={'input_tokens': 60, 'output_tokens': 17, 'total_tokens': 77, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token

In [34]:
tool_result = multiply.invoke(result.tool_calls[0])
messages.append(tool_result)

In [35]:
messages

[HumanMessage(content='Can you multiply 3 with 10', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 60, 'total_tokens': 77, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Chb5y3hondwokRDYHlW4SKvchzScB', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--c94f5d86-f688-421e-8123-f15bcb25583d-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 10}, 'id': 'call_eg3zlbbPrLNSEZodvarWtn7X', 'type': 'tool_call'}], usage_metadata={'input_tokens': 60, 'output_tokens': 17, 'total_tokens': 77, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token

In [36]:
llm_with_tools.invoke(messages).content

'The product of 3 and 10 is 30.'

Now just let's build a simple app to convert currency from US dollar to euro

In [ ]:
# tool create
from langchain_core.tools import InjectedToolArg
from typing import Annotated
@tool
def get_conversion_factor(base_currency: str, target_currency: str)->float:
    """This funtion fetches the currency conversion factor between a given base currency and a target currency"""
    
    url = "https://api.fastforex.io/fetch-one"
    API_KEY = '4498338605-2f3bc96886'
    params = {
        'from': base_currency, 
        'to': target_currency
    }
    headers = {
        "X-API-Key": API_KEY
    }
    
    response = requests.get(url, headers=headers, params=params, timeout=10)
    return response.json()


@tool
def convert(base_currency_value: int, conversion_rate: Annotated[float, InjectedToolArg] )->float:
    """Given a currency conversion rate this function calculates the target currency value from a given base currency value"""
    return base_currency_value * conversion_rate

In [70]:
get_conversion_factor.invoke({'base_currency': 'USD','target_currency': 'EUR'})

{'base': 'USD',
 'result': {'EUR': 0.8621},
 'updated': '2025-11-28T23:58:08Z',
 'ms': 8}

In [61]:
convert.invoke({'base_currency_value': 10, 'conversion_rate':0.8621})

8.621

In [62]:
# binding with llm

In [72]:
llm = ChatOpenAI()
llm_with_tools = llm.bind_tools([get_conversion_factor, convert])

In [88]:
messages = [HumanMessage('What is the convrsion factor between USD and EUR, and based on the conversion rate can you convert 20 usd to eur')]
messages

[HumanMessage(content='What is the convrsion factor between USD and EUR, and based on the conversion rate can you convert 20 usd to eur', additional_kwargs={}, response_metadata={})]

In [89]:
ai_message = llm_with_tools.invoke(messages)
messages.append(ai_message)

In [90]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'USD', 'target_currency': 'EUR'},
  'id': 'call_PV2afXRw4PzZYQHL5P4EBtum',
  'type': 'tool_call'},
 {'name': 'convert',
  'args': {'base_currency_value': 20},
  'id': 'call_gvr7KytPrZpMuEo2cEfIOgYT',
  'type': 'tool_call'}]

In [91]:
import json
for tool_call in ai_message.tool_calls:
    # execute the first toll and get value of converstion rate
    if tool_call['name'] == 'get_conversion_factor':
        tool_message1 = get_conversion_factor.invoke(tool_call)
        #print(tool_message1)
    # fetch this conversion rate
        conversion_rate = json.loads(tool_message1.content)['result']['EUR']
    # append this tool message to message list
        messages.append(tool_message1)
    # execute the 2nd tool using the conversion rate from tool 1
    if tool_call['name'] == 'convert':
        # fetch the current argument
        tool_call['args']['conversion_rate'] = conversion_rate
        tool_message2 = convert.invoke(tool_call)
        messages.append(tool_message2)

In [92]:
messages

[HumanMessage(content='What is the convrsion factor between USD and EUR, and based on the conversion rate can you convert 20 usd to eur', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 127, 'total_tokens': 178, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Chc2KVjlQ5DEObFAyLNuquXCCo95O', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--bd8e01b1-4fe8-4ae5-8081-68d5cec09005-0', tool_calls=[{'name': 'get_conversion_factor', 'args': {'base_currency': 'USD', 'target_currency': 'EUR'}, 'id': 'call_PV2afXRw4PzZYQHL5P4EBtum', 'type': 'tool_call'}, {'name': 'c

In [93]:
llm_with_tools.invoke(messages).content

'The conversion factor between USD and EUR is 0.8621.\n\nBased on this conversion rate, 20 USD is equivalent to approximately 17.242 EUR.'